# Affiliations report summary

In [1]:
from datetime import datetime
start = datetime.utcnow() # For measuring the total processing time

In [2]:
import json
from urllib.request import urlopen
import pandas as pd

## Get collection information from ArticleMeta 

In [3]:
AMC_URL = "http://articlemeta.scielo.org/api/v1/collection/identifiers/"
amc_data = pd.DataFrame(json.load(urlopen(AMC_URL)))

Some collections won't be analyzed, mainly to avoid duplicates
(there are articles in more than one collection).
The `spa` (*Public Health* collection) should have part of it
kept in the result, but it's not a collection
whose journals/articles are assigned to a single country.
The collections below are linked to a single country:

In [4]:
dont_evaluate = ["bio", "ecu", "psi", "rve", "rvt", "sss", "spa"]
amc_names_map = {
    "code": "collection",
    "acron2": "origin",
}
amc_pairs = amc_data \
    [(amc_data["acron2"].str.len() == 2) &
     ~amc_data["code"].isin(dont_evaluate)] \
    [list(amc_names_map.keys())] \
    .rename(columns=amc_names_map) \
    .assign(origin=lambda df: df["origin"].str.upper())
amc_pairs

,collection,origin
0,arg,AR
1,chl,CL
2,col,CO
3,cub,CU
4,esp,ES
5,mex,MX
6,prt,PT
7,scl,BR
10,sza,ZA
11,ven,VE


## ISSN selection from `spa`

These journals in the `spa` collection have the following countries:

In [5]:
spa_issn_country = pd.DataFrame([
    ("0021-2571", "IT"),
    ("0042-9686", "CH"),
    ("1020-4989", "US"),
    ("1555-7960", "US"),
], columns=["issn", "origin"])
spa_issn_country # For collection = "spa", only!

,issn,origin
0,0021-2571,IT
1,0042-9686,CH
2,1020-4989,US
3,1555-7960,US


## Affiliations dataset

This dataset is the
[Network spreadsheet/CSV pack](https://static.scielo.org/tabs/tabs_network.zip)
downloaded today (2018-08-09), which can be found in the
[SciELO Analytics report](https://analytics.scielo.org/w/reports)
web page.
The first two rows of it are:

In [6]:
dataset = pd.read_csv("documents_affiliations.csv", keep_default_na=False)
dataset.head(2).T

,0,1
extraction date,2018-06-10,2018-06-10
study unit,document,document
collection,scl,scl
ISSN SciELO,0100-879X,0100-879X
ISSN's,0100-879X;1414-431X,0100-879X;1414-431X
title at SciELO,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...
title thematic areas,Biological Sciences;Health Sciences,Biological Sciences;Health Sciences
title is agricultural sciences,0,0
title is applied social sciences,0,0
title is biological sciences,1,1


We won't need all the information,
and we can simplify the column names
for the columns we need:

In [7]:
names_map = {
    "document publishing ID (PID SciELO)": "pid",
    "document affiliation country ISO 3166": "country",
    "ISSN SciELO": "issn",
    "collection": "collection",
}
cdf = dataset[list(names_map.keys())].rename(columns=names_map)
cdf[610_000::80_000] # cdf stands for "Country/Collection Data Frame"

,pid,country,issn,collection
610000,S1516-18462014000501680,BR,1516-1846,scl
690000,S0100-879X2016001100703,BR,0100-879X,scl
770000,S1413-77042018000100041,BR,1413-7704,scl
850000,S1852-48342011000100014,BR,1852-4834,arg
930000,S0718-40262014000200005,CL,0718-4026,chl
1010000,S0375-07602015000100013,CU,0375-0760,cub
1090000,S0124-71072012000200007,,0124-7107,col
1170000,S0212-71992001001200007,,0212-7199,esp
1250000,S0034-83762005000500011,,0034-8376,mex
1330000,S1026-87742015000300361,,1026-8774,mex


## Adding journal country as `origin`

The `country` column in the last dataframe is the affiliation country,
not the journal/article origin country.
Let's add the former as a new `origin` column,
grabbing it from the collection
or from the ISSN (when collection is `spa`):

In [8]:
cdfwof = pd.concat([
    pd.merge(cdf[cdf["collection"] != "spa"], amc_pairs, how="inner", on="collection"),
    pd.merge(cdf[cdf["collection"] == "spa"], spa_issn_country, how="inner", on="issn"),
])
cdfwof[610_000::80_000] # wof stands for "With Origin, Filtered"

,pid,country,issn,collection,origin
610000,S1516-18462014000501680,BR,1516-1846,scl,BR
690000,S0100-879X2016001100703,BR,0100-879X,scl,BR
770000,S1413-77042018000100041,BR,1413-7704,scl,BR
850000,S1852-48342011000100014,BR,1852-4834,arg,AR
930000,S0718-40262014000200005,CL,0718-4026,chl,CL
1010000,S0375-07602015000100013,CU,0375-0760,cub,CU
1090000,S0124-71072012000200007,,0124-7107,col,CO
1170000,S1138-123X2002000700006,,1138-123X,esp,ES
1250000,S0187-62362007000300006,MX,0187-6236,mex,MX
1330000,S1026-87742015000300415,AR,1026-8774,mex,MX


The rows without an assignable origin have been removed:

In [9]:
cdf.shape

(1660403, 4)

In [10]:
cdfwof.shape

(1496340, 5)

## Country summary

Are the affiliations countries and the journal/origin country always the same?
The goal now is to create a summary of the affiliation countries
by comparing them to the journal/origin country. 

In [11]:
origin_country = cdfwof["country"] == cdfwof["origin"]
result = cdfwof.assign(
    origin_country=origin_country,
    other_country=~(origin_country | (cdfwof["country"] == "")),
    no_country=cdfwof["country"] == "",
).groupby("pid").sum().assign(
    has_origin=lambda df: df["origin_country"].apply(bool),
    has_other=lambda df: df["other_country"].apply(bool),
    has_no=lambda df: df["no_country"].apply(bool),
).assign(
    has_both=lambda df: df["has_origin"] & df["has_other"],
    all_no=lambda df: ~(df["has_origin"] | df["has_other"]),
).applymap(int)
result[:20_000:2_500]

,origin_country,other_country,no_country,has_origin,has_other,has_no,has_both,all_no
pid,,,,,,,,
S0001-37141998000300001,0,0,1,0,0,1,0,1
S0001-60022005000100009,0,0,1,0,0,1,0,1
S0003-25732007000400006,0,1,0,0,1,0,0,0
S0004-06222006000200011,0,1,0,0,1,0,0,0
S0004-27302011000400005,1,0,0,1,0,0,0,0
S0004-27492013000600013,3,0,0,1,0,0,0,0
S0004-282X1964000200003,1,0,0,1,0,0,0,0
S0004-282X1999000200020,0,0,1,0,0,1,0,1


Each row has an affiliation summary for a single article,
identified by its PID.
A brief explanation of the columns:

* `origin_country`: Number of affiliations whose country is the origin country;
* `other_country`: Number of affiliations whose country isn't the origin country;
* `no_country`: Number of affiliations whose country is unknown;
* `has_origin`: This article has at least one affiliation whose country is the origin country;
* `has_other`: This article has at least one affiliation whose country isn't the origin country;
* `has_no`: This article has at least one affiliation whose country is unknown;
* `has_both`: This article has affiliations from both the origin country and another country;
* `all_no`: All affiliations are from unknown countries.

The trailing columns are represented by the integers
`1` (meaning `True`) and `0` (meaning `False`).

## Final result

Let's join the ISSN, collection and origin information to our analysis:

In [12]:
full_result = \
    pd.merge(result.reset_index(),
             cdfwof[["pid", "issn", "collection", "origin"]].drop_duplicates(),
             how="left", on="pid") \
      .set_index("pid") \
      .sort_index()
full_result[7_500::30_000]

,origin_country,other_country,no_country,has_origin,has_other,has_no,has_both,all_no,issn,collection,origin
pid,,,,,,,,,,,
S0004-06222006000200011,0,1,0,0,1,0,0,0,0004-0622,ven,VE
S0034-70942005000300014,0,0,3,0,0,1,0,1,0034-7094,scl,BR
S0034-98872006000300022,0,0,1,0,0,1,0,1,0034-9887,chl,CL
S0066-782X2007001600001,1,1,0,1,1,0,1,0,0066-782X,scl,BR
S0100-29452013000200039,3,0,0,1,0,0,0,0,0100-2945,scl,BR
S0101-20612005000100012,1,0,1,1,0,1,0,0,0101-2061,scl,BR
S0102-33062015000200251,3,0,0,1,0,0,0,0,0102-3306,scl,BR
S0103-50532008000400012,1,0,0,1,0,0,0,0,0103-5053,scl,BR
S0104-12902016000200306,1,0,0,1,0,0,0,0,0104-1290,scl,BR


Saving the \[sorted\] result:

In [13]:
full_result.to_csv("documents_affiliations_country_summary.csv")

(The file has **41MB**, so it shouldn't be included in this git repository).

## Checking the result

There should be no more affiliations than what we had when we started... nor less...

In [14]:
full_result[["origin_country", "other_country", "no_country"]].values.sum() == cdfwof.shape[0]

True

In [15]:
print(f"Notebook processing duration: {datetime.utcnow() - start}")

Notebook processing duration: 0:00:15.909458
